In [1]:
import os
os.environ["SPARK_HOME"] = r"C:\Spark\spark-3.5.7-bin-hadoop3"
os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["JAVA_HOME"] = r"C:\Program Files\Eclipse Adoptium\jdk-17.0.17.10-hotspot"

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("pqt").master("local[3]").getOrCreate()
spark

In [4]:
std_prqt=spark.read.format('parquet').load(r"C:\Users\user1\pyspark_env\studentdata_nulls.parquet")

In [6]:
std_prqt.show()

+----------+------------+----+------+----------------+--------------+---------+-----+-----+
|student_id|student_name| age|gender|          course|admission_date|     city|grade|marks|
+----------+------------+----+------+----------------+--------------+---------+-----+-----+
|         1|   Student_1|NULL|Female|           Civil|    2022-09-30|Hyderabad|    C| 70.0|
|         2|   Student_2|18.0|Female|           Civil|    2023-03-21|Bangalore|    D| 60.0|
|         3|   Student_3|21.0|  Male|Computer Science|    2022-11-17|     Pune|    D| 70.0|
|         4|   Student_4|22.0|  Male|      Mechanical|    2022-10-09|    Delhi|    C| 80.0|
|         5|   Student_5|18.0|  NULL|      Electrical|    2023-02-20|Bangalore| NULL| 60.0|
|         6|   Student_6|21.0|Female|             ECE|    2022-07-20|     Pune|    B| 70.0|
|         7|   Student_7|21.0|  Male|              IT|    2022-11-09|Bangalore|    D| 80.0|
|         8|   Student_8|18.0|  Male|Computer Science|    2022-12-30|    Delhi| 

In [7]:
std1=std_prqt.na.fill({'gender':'O','age':0,'admission_date':'2025-11-07','grade':'F','marks':0})
std1.show()

+----------+------------+----+------+----------------+--------------+---------+-----+-----+
|student_id|student_name| age|gender|          course|admission_date|     city|grade|marks|
+----------+------------+----+------+----------------+--------------+---------+-----+-----+
|         1|   Student_1| 0.0|Female|           Civil|    2022-09-30|Hyderabad|    C| 70.0|
|         2|   Student_2|18.0|Female|           Civil|    2023-03-21|Bangalore|    D| 60.0|
|         3|   Student_3|21.0|  Male|Computer Science|    2022-11-17|     Pune|    D| 70.0|
|         4|   Student_4|22.0|  Male|      Mechanical|    2022-10-09|    Delhi|    C| 80.0|
|         5|   Student_5|18.0|     O|      Electrical|    2023-02-20|Bangalore|    F| 60.0|
|         6|   Student_6|21.0|Female|             ECE|    2022-07-20|     Pune|    B| 70.0|
|         7|   Student_7|21.0|  Male|              IT|    2022-11-09|Bangalore|    D| 80.0|
|         8|   Student_8|18.0|  Male|Computer Science|    2022-12-30|    Delhi| 

In [10]:
std2=std1.where(std1['age']>15).select(std1.student_id,std1.student_name,std1.age,std1.gender,std1.course,std1.grade)
std2.show()

+----------+------------+----+------+----------------+-----+
|student_id|student_name| age|gender|          course|grade|
+----------+------------+----+------+----------------+-----+
|         2|   Student_2|18.0|Female|           Civil|    D|
|         3|   Student_3|21.0|  Male|Computer Science|    D|
|         4|   Student_4|22.0|  Male|      Mechanical|    C|
|         5|   Student_5|18.0|     O|      Electrical|    F|
|         6|   Student_6|21.0|Female|             ECE|    B|
|         7|   Student_7|21.0|  Male|              IT|    D|
|         8|   Student_8|18.0|  Male|Computer Science|    B|
|        10|  Student_10|20.0|  Male|             ECE|    A|
|        11|  Student_11|21.0|     O|Computer Science|    F|
|        12|  Student_12|23.0|Female|           Civil|    F|
|        13|  Student_13|20.0|Female|              IT|    C|
|        14|  Student_14|20.0|Female|      Electrical|    C|
|        15|  Student_15|21.0|Female|      Mechanical|    A|
|        16|  Student_16

In [11]:
std2.count()

91

In [12]:
std1.count()

100

In [15]:
std1.crossJoin(std1).count()


10000

In [18]:
ijoin=std1.join(std2,std2['age']==std1['age'])

In [20]:
ijoin.show()
ijoin.count()

+----------+------------+----+------+----------------+--------------+---------+-----+-----+----------+------------+----+------+----------------+-----+
|student_id|student_name| age|gender|          course|admission_date|     city|grade|marks|student_id|student_name| age|gender|          course|grade|
+----------+------------+----+------+----------------+--------------+---------+-----+-----+----------+------------+----+------+----------------+-----+
|         2|   Student_2|18.0|Female|           Civil|    2023-03-21|Bangalore|    D| 60.0|        58|  Student_58|18.0|  Male|             ECE|    D|
|         2|   Student_2|18.0|Female|           Civil|    2023-03-21|Bangalore|    D| 60.0|        55|  Student_55|18.0|Female|      Mechanical|    F|
|         2|   Student_2|18.0|Female|           Civil|    2023-03-21|Bangalore|    D| 60.0|        50|  Student_50|18.0|Female|Computer Science|    B|
|         2|   Student_2|18.0|Female|           Civil|    2023-03-21|Bangalore|    D| 60.0|   

1461

In [22]:
ljoin=std1.join(std2,std2['course']==std1['course'],'left')
ljoin.count()
ljoin.show()

+----------+------------+----+------+------+--------------+---------+-----+-----+----------+------------+----+------+------+-----+
|student_id|student_name| age|gender|course|admission_date|     city|grade|marks|student_id|student_name| age|gender|course|grade|
+----------+------------+----+------+------+--------------+---------+-----+-----+----------+------------+----+------+------+-----+
|         1|   Student_1| 0.0|Female| Civil|    2022-09-30|Hyderabad|    C| 70.0|        95|  Student_95|19.0|  Male| Civil|    D|
|         1|   Student_1| 0.0|Female| Civil|    2022-09-30|Hyderabad|    C| 70.0|        93|  Student_93|22.0|  Male| Civil|    F|
|         1|   Student_1| 0.0|Female| Civil|    2022-09-30|Hyderabad|    C| 70.0|        92|  Student_92|22.0|  Male| Civil|    C|
|         1|   Student_1| 0.0|Female| Civil|    2022-09-30|Hyderabad|    C| 70.0|        86|  Student_86|21.0|  Male| Civil|    F|
|         1|   Student_1| 0.0|Female| Civil|    2022-09-30|Hyderabad|    C| 70.0|  

In [24]:
rjoin=std1.join(std2,std2['course']==std1['course'],'rightouter')
rjoin.show()

+----------+------------+----+------+----------------+--------------+---------+-----+-----+----------+------------+----+------+----------------+-----+
|student_id|student_name| age|gender|          course|admission_date|     city|grade|marks|student_id|student_name| age|gender|          course|grade|
+----------+------------+----+------+----------------+--------------+---------+-----+-----+----------+------------+----+------+----------------+-----+
|        99|  Student_99| 0.0|Female|           Civil|    2023-03-16|     NULL|    D| 80.0|         2|   Student_2|18.0|Female|           Civil|    D|
|        95|  Student_95|19.0|  Male|           Civil|    2022-08-17|Bangalore|    D| 70.0|         2|   Student_2|18.0|Female|           Civil|    D|
|        93|  Student_93|22.0|  Male|           Civil|    2023-01-22|Bangalore|    F| 90.0|         2|   Student_2|18.0|Female|           Civil|    D|
|        92|  Student_92|22.0|  Male|           Civil|    2025-11-07|  Chennai|    C| 70.0|   

In [25]:
rjoin.count()

1526

In [26]:
fjoin=std1.join(std2,std2['course']==std1['course'],'full_outer')
fjoin.count()

1526

In [27]:
sjoin=std1.join(std2,std2['course']==std1['course'],'semi')
sjoin.count()

100

In [28]:
Ajoin=std1.join(std2,std2['course']==std1['course'],'anti')
Ajoin.count()

0

In [29]:
Ajoin.show()


+----------+------------+---+------+------+--------------+----+-----+-----+
|student_id|student_name|age|gender|course|admission_date|city|grade|marks|
+----------+------------+---+------+------+--------------+----+-----+-----+
+----------+------------+---+------+------+--------------+----+-----+-----+



In [31]:
from pyspark.sql.types import *

In [32]:
sch=StructType([\
    StructField("Name",StringType(),True),
    StructField("Department",StringType(),True),
    StructField("Salary",IntegerType(),True),
])

In [33]:
data=spark.read.format('csv').schema(sch).load(r"C:\Users\user1\pyspark_env\windowdata.txt")
#lazyexecution

In [34]:
data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: integer (nullable = true)



In [36]:
data.show()

+------+----------+------+
|  Name|Department|Salary|
+------+----------+------+
|  John|        HR| 45000|
| David|        HR| 48000|
| Maria|        HR| 52000|
|  Arun|        IT| 60000|
| Kavya|        IT| 65000|
| Meena|        IT| 60000|
|   Raj|   Finance| 55000|
|  Rita|   Finance| 58000|
| Vijay|   Finance| 62000|
|Suresh|     Sales| 50000|
| Priya|     Sales| 55000|
|  Neha|     Sales| 58000|
+------+----------+------+



In [38]:
res=data.groupby('Department').max('Salary')
res.show()

+----------+-----------+
|Department|max(Salary)|
+----------+-----------+
|     Sales|      58000|
|        HR|      52000|
|   Finance|      62000|
|        IT|      65000|
+----------+-----------+



In [40]:
from pyspark.sql.window import *
from pyspark.sql.functions import *

In [44]:
wi1=Window.partitionBy('Department')

In [49]:
res1=data.withColumn("mxsal",max('Salary').over(wi1))
res1.show()

+------+----------+------+-----+
|  Name|Department|Salary|mxsal|
+------+----------+------+-----+
|   Raj|   Finance| 55000|62000|
|  Rita|   Finance| 58000|62000|
| Vijay|   Finance| 62000|62000|
|  John|        HR| 45000|52000|
| David|        HR| 48000|52000|
| Maria|        HR| 52000|52000|
|  Arun|        IT| 60000|65000|
| Kavya|        IT| 65000|65000|
| Meena|        IT| 60000|65000|
|Suresh|     Sales| 50000|58000|
| Priya|     Sales| 55000|58000|
|  Neha|     Sales| 58000|58000|
+------+----------+------+-----+



In [47]:
res1=data.withColumn("dfsal",max('Salary').over(wi1) - col('Salary'))
res1.show()

+------+----------+------+-----+
|  Name|Department|Salary|dfsal|
+------+----------+------+-----+
|   Raj|   Finance| 55000| 7000|
|  Rita|   Finance| 58000| 4000|
| Vijay|   Finance| 62000|    0|
|  John|        HR| 45000| 7000|
| David|        HR| 48000| 4000|
| Maria|        HR| 52000|    0|
|  Arun|        IT| 60000| 5000|
| Kavya|        IT| 65000|    0|
| Meena|        IT| 60000| 5000|
|Suresh|     Sales| 50000| 8000|
| Priya|     Sales| 55000| 3000|
|  Neha|     Sales| 58000|    0|
+------+----------+------+-----+



In [51]:
win2=Window.partitionBy('Department').orderBy('Salary')

In [52]:
res2=data.withColumn('rnum',row_number().over(win2))
res2.show()

+------+----------+------+----+
|  Name|Department|Salary|rnum|
+------+----------+------+----+
|   Raj|   Finance| 55000|   1|
|  Rita|   Finance| 58000|   2|
| Vijay|   Finance| 62000|   3|
|  John|        HR| 45000|   1|
| David|        HR| 48000|   2|
| Maria|        HR| 52000|   3|
|  Arun|        IT| 60000|   1|
| Meena|        IT| 60000|   2|
| Kavya|        IT| 65000|   3|
|Suresh|     Sales| 50000|   1|
| Priya|     Sales| 55000|   2|
|  Neha|     Sales| 58000|   3|
+------+----------+------+----+



In [53]:
res2=data.withColumn('rnk',rank().over(win2))
res2.show()

+------+----------+------+---+
|  Name|Department|Salary|rnk|
+------+----------+------+---+
|   Raj|   Finance| 55000|  1|
|  Rita|   Finance| 58000|  2|
| Vijay|   Finance| 62000|  3|
|  John|        HR| 45000|  1|
| David|        HR| 48000|  2|
| Maria|        HR| 52000|  3|
|  Arun|        IT| 60000|  1|
| Meena|        IT| 60000|  1|
| Kavya|        IT| 65000|  3|
|Suresh|     Sales| 50000|  1|
| Priya|     Sales| 55000|  2|
|  Neha|     Sales| 58000|  3|
+------+----------+------+---+



In [54]:
res2=data.withColumn('drnk',dense_rank().over(win2))
res2.show()

+------+----------+------+----+
|  Name|Department|Salary|drnk|
+------+----------+------+----+
|   Raj|   Finance| 55000|   1|
|  Rita|   Finance| 58000|   2|
| Vijay|   Finance| 62000|   3|
|  John|        HR| 45000|   1|
| David|        HR| 48000|   2|
| Maria|        HR| 52000|   3|
|  Arun|        IT| 60000|   1|
| Meena|        IT| 60000|   1|
| Kavya|        IT| 65000|   2|
|Suresh|     Sales| 50000|   1|
| Priya|     Sales| 55000|   2|
|  Neha|     Sales| 58000|   3|
+------+----------+------+----+



In [55]:
res2.write.mode("overwrite").parquet(r"C:\Users\user1\pyspark_env\window_result\transfrmation.parquet")

In [56]:
res2.write.mode("overwrite").json(r"C:\Users\user1\pyspark_env\jsonfiles\transfrmation.json")

In [57]:
#Full Refresh Load= deleting or overwriting all existing data in the target table or files sytem , then load entire source data again
#incremental load=load only new or chhanges records 

#year-2024--month=12
            #month=11

#year-2025 

#res2.write.mode("overwrite").partionBy("yaer","month").json("url"))